<a href="https://colab.research.google.com/github/Elenacola/analisi_data/blob/main/Data_Analytics_autovetture%20Milano_video%20esercitazione%20Boolean%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Analytics in Python

Questo che stai vedendo è un **Python Notebook** creato usando **Google Colaboratory**, puoi usarlo così com'è senza bisogno di installare nulla e puoi interagire con il codice che vedi più sotto cliccando sul tasto ▶ che vedi nelle celle contenenti il codice (se non vedi il tasto, prova a muovere il mouse sopra alla cella). 

Se vuoi puoi creare un Notebook tutto tuo partendo dal tuo Google Drive e usando il tasto ➕ per aggiungere un nuovo documento *Google Colaboratory*. A [questo link](https://colab.research.google.com/) troverai maggiori informazioni per iniziare. 

<br>

In questo breve notebook introduttivo voglio mostrarti un **esempio di analisi** che potresti fare durante il corso e nella tua futura carriera nel mondo della Data Analytics. 

Analizzeremo dei **dati presi dal portale del Comune di Milano**, in particolare: 


* *Numero di autovetture per classe ambientale dal 2007 al 2020* 
* *Valori rilevati per i principali inquinanti dell'aria dal 2004 al 2018* 

Lo scopo finale dell'analisi sarà quello di **creare un modello** che metta in relazione il numero di autovetture in alta classe ambientale con la concentrazione di polveri sottili nell'aria, in modo tale da poter usare una variabile per **prevedere** l'altra. 

L'analisi è divisa come segue: 


*   Setup iniziale e caricamento delle librerie necessarie
*   Caricamento dei dati
*   Analisi descrittiva dei dati
*   Analisi grafica dei dati
*   Manipolazione e unione dei dati
*   Analisi di regressione lineare


<br>

---

## Setup iniziale

In [ ]:
# Importo le librerie necessarie
from sklearn.linear_model import LinearRegression
import plotly.express as px
import pandas as pd
import numpy as np
import requests
import json

<br>

---


## Autovetture per classe ambientale dal 2007 al 2020

Mi connetto al portale dati del Comune di Milano e scarico una tabella contenente il numero di autovetture per classe ambientale dal 2007 al 2020 ([link](https://dati.comune.milano.it/dataset/ds1081-autovetture-dei-residenti-a-milano-suddivise-per-classe-ambientale-anno-serie-storica)).

In [ ]:
url = 'https://dati.comune.milano.it/api/3/action/datastore_search?resource_id=05398947-5707-49b0-a1d1-ee494b41543c&limit=500'
response = requests.get(url)
json = response.json()

Stampo a schermo la tabella che ho appena scaricato dopo averla convertita in un formato, chiamato DataFrame, che ne semplifica la manipolazione in Python.


In [ ]:
df_auto = pd.DataFrame(json['result']['records'])
df_auto.head()

,_id,ANNO,CLASSE AMBIENTALE,AUTOVETTURE
0,1,2007,EURO 0,95734.0
1,2,2007,EURO 1,56369.0
2,3,2007,EURO 2,182124.0
3,4,2007,EURO 3,178629.0
4,5,2007,EURO 4,213722.0


Elimino la variabile id che non mi serve e rinomino le colonne.

In [ ]:
df_auto = df_auto.drop(columns=['_id'])
df_auto.columns = df_auto.columns.str.lower()
df_auto = df_auto.rename(columns={'classe ambientale':'classe_ambientale'})
df_auto.head()

,anno,classe_ambientale,autovetture
0,2007,EURO 0,95734.0
1,2007,EURO 1,56369.0
2,2007,EURO 2,182124.0
3,2007,EURO 3,178629.0
4,2007,EURO 4,213722.0


Faccio una prima analisi descrittiva del mio dataset.

In [ ]:
print('Numero di righe e colonne: ')
df_auto.shape

Numero di righe e colonne: 


(124, 3)

In [ ]:
print('Tipi di variabili nel DataFrame: ')
df_auto.dtypes

Tipi di variabili nel DataFrame: 


anno                   int64
classe_ambientale     object
autovetture          float64
dtype: object

In [ ]:
print('Anni a disposizione: ')
list(df_auto.anno.unique())

Anni a disposizione: 


[2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]

In [ ]:
print('Elenco delle classi ambientali presenti: ')
list(df_auto.classe_ambientale.unique())

Elenco delle classi ambientali presenti: 


['EURO 0',
 'EURO 1',
 'EURO 2',
 'EURO 3',
 'EURO 4',
 'EURO 5',
 'EURO 6',
 'Non contemplato',
 'Non identificato',
 'Non definito',
 'non definito',
 'Euro 0',
 'Euro 1',
 'Euro 2',
 'Euro 3',
 'Euro 4',
 'Euro 5',
 'Euro 6']

In [ ]:
print('Analisi descrittiva della variabile autovetture: ')
df_auto.autovetture.describe()

Analisi descrittiva della variabile autovetture: 


count       119.000000
mean      82879.218487
std       79449.632694
min           0.000000
25%        9760.000000
50%       75385.000000
75%      137098.500000
max      308481.000000
Name: autovetture, dtype: float64

In [ ]:
print('Visualizzo le righe contenenti dei valori mancanti: ')
df_auto[df_auto.autovetture.isna()==True]

Visualizzo le righe contenenti dei valori mancanti: 


,anno,classe_ambientale,autovetture
5,2007,EURO 5,NaN
6,2007,EURO 6,NaN
15,2008,EURO 6,NaN
24,2009,EURO 6,NaN
33,2010,EURO 6,NaN


Filtro il DataFrame e tengo solo le righe con un valore di classe ambientale EURO# (elimino le categorie 'non contemplato', 'non definito', ecc).

In [ ]:
# Rendo tutto uppercase per omogeneizzarei dati (negli ultimi anni EURO è scritto in minuscolo)
df_auto.classe_ambientale = df_auto.classe_ambientale.str.upper()
# Tengo solo ciò che contiene la dicitura 'EURO'
df_auto = df_auto[df_auto.classe_ambientale.str.contains('EURO')]
df_auto.head()

,anno,classe_ambientale,autovetture
0,2007,EURO 0,95734.0
1,2007,EURO 1,56369.0
2,2007,EURO 2,182124.0
3,2007,EURO 3,178629.0
4,2007,EURO 4,213722.0


Usando un **grafico a linee** (line chart), visualizzo la variabile autovetture per classe_ambientale dal 2007 al 2020.

In [ ]:
fig = px.line(df_auto, x='anno', y='autovetture', color='classe_ambientale', 
              template='plotly', title="Line Chart | Numero di autovetture per classe ambientale dal 2007 al 2020")
fig.update_traces(line=dict(width=3))
fig.update_layout(
    xaxis_title="Anno",
    yaxis_title="Nr. autovetture")
fig.show()

Usando un altro tipo di visualizzazione, il **grafico ad area** (area chart), scopriamo che il numero totale di autovetture è rimasto stabile negli ultimi anni, ma la composizione per classe ambientale è cambiata notevolmente.

In [ ]:
fig = px.area(df_auto, x='anno', y='autovetture', color='classe_ambientale', 
              template='plotly', title="Area Chart | Numero di autovetture per classe ambientale dal 2007 al 2020")
fig.update_traces(line=dict(width=3))
fig.update_layout(
    xaxis_title="Anno",
    yaxis_title="Nr. autovetture")
fig.show()

## Principali inquinanti dell'aria

Mi connetto al portale dati del Comune di Milano e scarico una tabella contenente i dati legati all'inquinamento dell'aria dal 2004 al 2018 ([link](https://dati.comune.milano.it/dataset/ds573-valori-rilevati-per-i-principali-inquinanti-dell-aria)).

In [ ]:
url = 'https://dati.comune.milano.it/api/3/action/datastore_search?resource_id=82efde46-5354-414b-8826-ba6ada03ac8b&limit=500'
response = requests.get(url)
# Estraggo il contenuto dei dati in JSON
json = response.json()
df_air = pd.DataFrame(json['result']['records'])
df_air.head()

,_id,anno_rilevamento_inquinanti_aria,inquinanti_aria_tipologia,inquinanti_aria_indicatori,inquinanti_aria
0,1,2018,Polveri sottili - PM10,Media annua,35
1,2,2018,Polveri sottili - PM10,Giorni di superamento limite giornaliero,79
2,3,2018,Biossido di azoto - NO2,Media annua,59
3,4,2018,Ozono - O3,Giorni di superamento limite giornaliero,56
4,5,2017,Polveri sottili - PM10,Media annua,40


Elimino la variabile id, rinomino le colonne del DataFrame e tengo solo le righe con la media annua.

In [ ]:
df_air = df_air.drop(columns=['_id'])
df_air = df_air.rename(columns={'anno_rilevamento_inquinanti_aria':'anno', 
                       'inquinanti_aria_tipologia':'inquinante_tipo', 
                       'inquinanti_aria_indicatori':'inquinante_indicatore', 
                       'inquinanti_aria':'inquinante_valore'})
df_air = df_air[df_air.inquinante_indicatore=='Media annua']
df_air.head()

,anno,inquinante_tipo,inquinante_indicatore,inquinante_valore
0,2018,Polveri sottili - PM10,Media annua,35
2,2018,Biossido di azoto - NO2,Media annua,59
4,2017,Polveri sottili - PM10,Media annua,40
6,2017,Biossido di azoto - NO2,Media annua,64
8,2016,Polveri sottili - PM10,Media annua,"36,3"


Faccio una prima analisi descrittiva del mio dataset.

In [ ]:
print('Numero di righe e colonne: ')
df_air.shape

Numero di righe e colonne: 


(30, 4)

In [ ]:
print('Tipi di variabili nel DataFrame: ')
df_air.dtypes

Tipi di variabili nel DataFrame: 


anno                      int64
inquinante_tipo          object
inquinante_indicatore    object
inquinante_valore        object
dtype: object

In [ ]:
# Converto la variabile anno in un int e nella variabile inquinante_valore sostituisco , con un . per il separatore 
# delle decine (crea problemi) e converto a valore decimale:
df_air.inquinante_valore = df_air.inquinante_valore.str.replace(',','.')
df_air = df_air.astype({"anno": int, "inquinante_valore": float})

In [ ]:
print('Anni a disposizione: ')
list(df_air.anno.unique())

Anni a disposizione: 


[2018,
 2017,
 2016,
 2015,
 2014,
 2013,
 2012,
 2011,
 2010,
 2009,
 2008,
 2007,
 2006,
 2005,
 2004]

In [ ]:
print('Elenco delle tipologie di inquinanti: ')
list(df_air.inquinante_tipo.unique())

Elenco delle tipologie di inquinanti: 


['Polveri sottili - PM10', 'Biossido di azoto - NO2']

In [ ]:
print('Analisi descrittiva della variabile inquinante_valore: ')
df_air.inquinante_valore.describe()

Analisi descrittiva della variabile inquinante_valore: 


count    30.000000
mean     50.646667
std       9.521436
min      34.000000
25%      43.425000
50%      51.350000
75%      58.775000
max      67.400000
Name: inquinante_valore, dtype: float64

Visualizzo la concentrazione per tipo di agente inquinante nel tempo. Possiamo notare un trend decrescente dei valori di PM10 nel tempo.

In [ ]:
fig = px.line(df_air, x='anno', y='inquinante_valore', color='inquinante_tipo', 
              template='plotly', title="Line Chart | Concentrazione dell'inquinante (µg/m3) per tipologia dal 2004 al 2018")
fig.update_traces(line=dict(width=3))
fig.update_layout(
    xaxis_title="Anno",
    yaxis_title="Concentrazione (µg/m3)")
fig.show()

Unisco i due DataFrame e li "ristrutturo" in modo tale da avere un anno per ogni riga e tutte le variabili in colonna.

In [ ]:
# prima ristrutturazione dei due dataset da "long" a "wide"
df_auto_wide = df_auto.pivot(index='anno', columns='classe_ambientale', values='autovetture')
df_air_wide = df_air.pivot(index='anno', columns='inquinante_tipo', values='inquinante_valore')
# left join di df_auto su df_air
df_all = df_auto_wide.join(df_air_wide, on='anno', how='left')
# pulisco i nomi delle colonne per renderli più leggibili
df_all.columns = df_all.columns.str.replace(" ", "")
df_all = df_all.rename(columns={'Biossidodiazoto-NO2':'NO2', 'Polverisottili-PM10':'PM10'})
df_all = df_all.drop(columns=['NO2']).fillna(0)
df_all

,EURO0,EURO1,EURO2,EURO3,EURO4,EURO5,EURO6,PM10
anno,,,,,,,,
2007,95734.0,56369.0,182124.0,178629.0,213722.0,0.0,0.0,51.0
2008,90066.0,45979.0,160933.0,162922.0,263553.0,0.0,0.0,44.7
2009,86185.0,37299.0,135834.0,150576.0,291036.0,14977.0,0.0,45.0
2010,83593.0,31617.0,116427.0,138363.0,308481.0,37418.0,0.0,39.7
2011,82529.0,28067.0,103387.0,128149.0,277675.0,104173.0,173.0,49.0
2012,81084.0,25269.0,91749.0,117272.0,261126.0,138913.0,405.0,43.0
2013,79521.0,22609.0,82652.0,107618.0,247485.0,149360.0,11604.0,37.3
2014,78765.0,20655.0,73239.0,99246.0,234046.0,172062.0,7916.0,34.0
2015,77460.0,19375.0,66156.0,92302.0,224268.0,183983.0,22613.0,40.0


<br>

---

## Regressione Lineare

Come possiamo vedere dall'ultimo DataFrame, **mancano i valori di PM10 per gli anni 2019 e 2020**. Ci sono diversi modi per risolvere il problema dei dati mancanti, il primo tra tutti è recuperare altri dati se possibile, altrimenti  esistono vari metodi di interpolazione e previsione più o meno sofisticati. 

Il metodo che qui andremo a usare a scopo illustrativo è quello di creare un modello di [Regressione Lineare](https://it.wikipedia.org/wiki/Regressione_lineare) per prevedere i valori mancanti. In pratica **andremo a costruire un modello** che, dato il numero di autovetture di nuova generazione (EURO 4, 5 e 6) sia in grado di prevedere la concentrazione media di polveri sottili nell'aria. 

Per prima cosa, creo una copia del DataFrame dove tengo una sola variabile autovetture pari alla somma delle EURO 4, 5 e 6.

In [ ]:
df_456 = df_all.copy()
df_456['EURO456'] = df_456['EURO4'] + df_456['EURO5'] + df_456['EURO6']
df_456.reset_index(inplace=True)
df_456 = df_456[['anno', 'EURO456', 'PM10']]
df_456

,anno,EURO456,PM10
0,2007,213722.0,51.0
1,2008,263553.0,44.7
2,2009,306013.0,45.0
3,2010,345899.0,39.7
4,2011,382021.0,49.0
5,2012,400444.0,43.0
6,2013,408449.0,37.3
7,2014,414024.0,34.0
8,2015,430864.0,40.0
9,2016,451694.0,36.3


Vado quindi a creare un **grafico di dispersione** (scatter plot) dove metto in relazione le due variabili (autovetture EURO456 e inquinamento PM10). Questo ci permette di vedere subito una **relazione inversa** tra le due, cioè la concentrazionedi PM10 scende al salire del numero di autovetture EURO456.

In [ ]:
x = df_456.EURO456
y = df_456.PM10
# dato che mancano i valori per PM10 e voglio creare un modello che preveda 
# proprio questi due punti, rimuovo le ultime due osservazioni 
x2 = x.drop(x.tail(2).index)
y2 = y.drop(y.tail(2).index)
# Visualizzo lo scatter plot
fig = px.scatter(x=x2, y=y2)
fig.update_traces(marker_size=10)
fig.update_layout(
    title="Scatter Plot | Relazione tra Nr. Autovetture VS Concentrazione PM10",
    xaxis_title="Nr. autovetture EURO 4, 5 e 6",
    yaxis_title="Concentrazione PM10 (µg/m3)")
fig.show()

Definisco il modello di regressione lineare e lo eseguo sui dati a disposizione. Poi stampo a schermo l'indice di bontà del modello. 

In [ ]:
model = LinearRegression().fit(np.array(x2).reshape(-1,1), y2)
r_2 = model.score(np.array(x2).reshape(-1,1), y2)
print('Bontà del modello (R2):', r_2)

Bontà del modello (R2): 0.5583598772252887


A questo punto posso usare il modello per prevedere gli ultimi due valori mancanti di PM10...

In [ ]:
preds = model.predict(np.array(x.tail(2)).reshape(-1,1))
preds

array([35.64361681, 35.28492502])

... e aggiornare il DataFrame con i nuovi valori previsti.

In [ ]:
df_456['pred'] = 0.0
df_456.pred.to_numpy()[-2:] = preds
df_456.tail()

,anno,EURO456,PM10,pred
9,2016,451694.0,36.3,0.000000
10,2017,477964.0,40.0,0.000000
11,2018,486779.0,35.0,0.000000
12,2019,499355.0,0.0,35.643617
13,2020,506877.0,0.0,35.284925


Infine creo un nuovo scatter plot che mostri i valori vecchi, i due nuovi punti che abbiamo previsto e la relativa retta di regressione.

In [ ]:
# Ristrutturo il dataframe per creare lo scatterplot con colori diversi
df_456_long = df_456.melt(id_vars=['anno', 'EURO456'])
df_456_long.replace(0, np.nan, inplace=True)
# Visualizziamo lo scatter
import plotly.graph_objects as go

fig = px.scatter(df_456_long, x='EURO456', y='value', color='variable')
fig.update_traces(marker_size=10)
fig.add_trace(
    go.Scatter(x=df_456.EURO456, y=model.intercept_ + model.coef_[0]*df_456.EURO456, 
               name="regression line", line_shape='linear', 
               line=go.scatter.Line(color="gray"), opacity=0.6)
)
fig.update_layout(
    title="Scatter Plot | Relazione tra Nr. Autovetture VS Concentrazione PM10",
    xaxis_title="Nr. autovetture EURO 4, 5 e 6",
    yaxis_title="Concentrazione PM10 (µg/m3)")
fig.show()